# RoBERTa Regression

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import torch
from collections import defaultdict, Counter
import random
import math
import pickle

import src.eval_metric
import src.model
import src.dataloader

%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [14]:
train_df = pd.read_csv("../../data/training_data/train.csv")
valid_df = pd.read_csv("../../data/training_data/dev.csv")
test_df = pd.read_csv("../../data/test_data_subtask2/sub2/test_copy.csv")

In [3]:
def seperateHyphenToSentence(s):
    parts = s.split('-')
    return parts[-1]
def getLangText(s):
    parts = s.split('-')
    return '-'.join(parts[:-1])

In [4]:
train_df['langText'] = train_df.sentence_id.apply(getLangText).astype(str)
valid_df['langText'] = valid_df.sentence_id.apply(getLangText).astype(str)
test_df['langText'] = test_df.sentence_id.apply(getLangText).astype(str)

In [15]:
train_df['sentence_id'] = pd.factorize(train_df.sentence_id)[0] 
valid_df['sentence_id'] = pd.factorize(valid_df.sentence_id)[0] 

In [16]:
valid_df.tail()

,language,sentence_id,word_id,word,FFDAvg,FFDStd,TRTAvg,TRTStd
1544,nl,102,41.0,wasMet,11.353516,0.520182,17.778184,9.606035
1545,nl,103,1.0,U,12.765262,4.057740,21.817248,12.173254
1546,nl,103,2.0,wilt,11.580342,4.493241,15.669324,6.043732
1547,nl,103,3.0,niet,10.348129,3.490696,11.485933,4.261039
1548,nl,103,4.0,spreken?,9.457177,3.943935,10.094739,5.087791


## Fine-tune model

In [17]:
import os
PRED_DIR = '../../data/task2_predictions/'

In [11]:
model_trainer = src.model.ModelTrainerNew()
model_trainer.train(train_df, valid_df, num_epochs=50,batch_size = 16)
predict_df = model_trainer.test(test_df)
predict_df.to_csv(PRED_DIR+"predictions_raw.csv", index=False)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1
MAE for FFDAvg: 6.1910010529538635
MAE for FFDStd: 2.3922250632986244
MAE for TRTAvg: 9.298877343304254
MAE for TRTStd: 5.414986576571236
Overall MAE: 5.824272509031994
Epoch: 2
MAE for FFDAvg: 5.97004458625873
MAE for FFDStd: 2.563052424660061
MAE for TRTAvg: 8.889530168800226
MAE for TRTStd: 5.711820415268701
Overall MAE: 5.783611898746929
Epoch: 3
MAE for FFDAvg: 4.412797070936794
MAE for FFDStd: 2.0943542734212275
MAE for TRTAvg: 7.707288845538965
MAE for TRTStd: 4.916624766203072
Overall MAE: 4.7827662390250145
Epoch: 4
MAE for FFDAvg: 3.865894035578442
MAE for FFDStd: 2.009205682674153
MAE for TRTAvg: 7.101074165252214
MAE for TRTStd: 4.871289151778243
Overall MAE: 4.461865758820763
Epoch: 5
MAE for FFDAvg: 3.3015762514522398
MAE for FFDStd: 1.916108703904976
MAE for TRTAvg: 6.4523707679402875
MAE for TRTStd: 4.675680602929901
Overall MAE: 4.086434081556851
Epoch: 6
MAE for FFDAvg: 2.8503106915565866
MAE for FFDStd: 1.904771292658685
MAE for TRTAvg: 6.016439683129154
MAE

In [18]:
for text_name in text_names[6:]:
    print(text_name)
    model_trainer = src.model.ModelTrainer(text_name=text_name)
    model_trainer.train(train_df, valid_df, num_epochs=10,batch_size = 8)
    test_df = test_df[test_df.langText == 'CopCo']
    predict_df = model_trainer.test(test_df)
    predict_df.to_csv(PRED_DIR+text_name+"_predictions.csv", index=False)

PoTeC


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1
MAE for FFDAvg: 0.5823010735797874
MAE for FFDStd: 0.7400588015153581
MAE for TRTAvg: 10.526837360074616
MAE for TRTStd: 6.175311681908394
Overall MAE: 4.506127229269539
Epoch: 2
MAE for FFDAvg: 0.632071892499246
MAE for FFDStd: 0.5072833894676236
MAE for TRTAvg: 7.321183295523286
MAE for TRTStd: 4.5133615192632535
Overall MAE: 3.243475024188352
Epoch: 3
MAE for FFDAvg: 0.5469121296176844
MAE for FFDStd: 0.4815761183732746
MAE for TRTAvg: 6.133230318745308
MAE for TRTStd: 4.259392664237189
Overall MAE: 2.855277807743364
Epoch: 4
MAE for FFDAvg: 0.5014003681816833
MAE for FFDStd: 0.4826375896194039
MAE for TRTAvg: 5.910082573044346
MAE for TRTStd: 4.088316981782601
Overall MAE: 2.745609378157009
Epoch: 5
MAE for FFDAvg: 0.5087026832107618
MAE for FFDStd: 0.4815307669571876
MAE for TRTAvg: 5.797355044190964
MAE for TRTStd: 4.061848531678238
Overall MAE: 2.7123592565092878
Epoch: 6
MAE for FFDAvg: 0.5746192211749244
MAE for FFDStd: 0.5599158476712045
MAE for TRTAvg: 5.76013220215

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1
MAE for FFDAvg: 1.857463881204624
MAE for FFDStd: 1.4763231496938045
MAE for TRTAvg: 3.69542256427636
MAE for TRTStd: 2.8979468367944095
Overall MAE: 2.4817891079922996
Epoch: 2
MAE for FFDAvg: 1.7490215771672422
MAE for FFDStd: 1.471343104906308
MAE for TRTAvg: 3.3834930639685687
MAE for TRTStd: 2.7978862064200034
Overall MAE: 2.3504359881155303
Epoch: 3
MAE for FFDAvg: 1.7382563450885078
MAE for FFDStd: 1.4705238698309642
MAE for TRTAvg: 3.3679391506895064
MAE for TRTStd: 2.7861865163825845
Overall MAE: 2.3407264704978905
Epoch: 4
MAE for FFDAvg: 1.8073718039293414
MAE for FFDStd: 1.4941867413130827
MAE for TRTAvg: 3.3623832248729206
MAE for TRTStd: 2.8424449148450828
Overall MAE: 2.376596671240107
Epoch: 5
MAE for FFDAvg: 1.7604131126502887
MAE for FFDStd: 1.4810547185357577
MAE for TRTAvg: 3.355635063604232
MAE for TRTStd: 2.824189583930291
Overall MAE: 2.3553231196801425
Epoch: 6
MAE for FFDAvg: 1.7486858452350065
MAE for FFDStd: 1.470820808309667
MAE for TRTAvg: 3.406022

## Make predictions